# Introduction

The purpose of the tutorial is to demonstrate interactive visualization with solvation_analysis and [nglview](https://github.com/nglviewer/nglview). Please work through the basics tutorial first if you are unfamiliar with solvation_analysis.

In [37]:
# imports
import MDAnalysis as mda
from MDAnalysis import transformations
from solvation_analysis.solution import Solution
from solvation_analysis.tests import datafiles

# instantiate Universe
u = mda.Universe(datafiles.ec_emc_pdb, datafiles.ec_emc_dcd)

The setup here should be familiar. We define all of our atom groups of interest and use them to instantiate a solution. EC and EMC have similar compositions, so we select them by slicing the array of residues.

In [74]:
# define solute AtomGroup
li_atoms = u.atoms.select_atoms("element Li")

# define solvent AtomGroups
EA = u.residues[0:235].atoms                    # ethyl acetate
FEC = u.residues[235:600].atoms                 # fluorinated ethylene carbonate
PF6 = u.atoms.select_atoms("byres element P")   # hexafluorophosphate

# instantiate solution
solution = Solution(li_atoms, 
                    {'EA': EA, 'FEC': FEC, 'PF6': PF6}, 
                    radii={'PF6': 2.6, 'FEC': 2.7})

solution.run()

/Users/orioncohen/projects/development/solvation-analysis/solvation_analysis/solvation_analysis/solution.py:208: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  pairs[:, 1] = solvent.ix[[pairs[:, 1]]]


## Primer on nglview

nglview is a powerful interactive molecular visualization package. Unfortunately, it can be a bit of hassle to get working properly. Make sure you have ipywidgets installed before working with nglview.

For more information on nglview, check out [their website](http://nglviewer.org/nglview/latest/) or the [MDAnalysis nglview tutorial](https://www.mdanalysis.org/2016/03/14/nglview/).

The cell below provides a test case for your Jupyter notebook configuration. If everything is configured properly, the cell will print a rectangle with "Hello, World" printed inside. If your Jupyter environment is configured incorrectly, it will instead print `Text(value='Hello, World')`.

In [75]:
# import nglview
import nglview as nv

# test Jupyter configuration
from ipywidgets import Text
Text("Hello, World")

Text(value='Hello, World')

Now the exciting part, visualizing! We will start with a simple visualization of all atoms.

First, we call `nv.show_mdanalysis()` on an `AtomGroup` to instantiate an mda_view, this contains a persistent view of an MDA `AtomGroup`. Then, we call `mda_view.display()`, which should display a box of atoms below the cell.

In [76]:
mda_view = nv.show_mdanalysis(u.atoms)
mda_view.display()

NGLWidget(max_frame=9)

For convenience, I recommend wrapping this into a one-liner function, shown below. This is how we will use `nglview` for the remainder of the tutorial.

In [77]:
def visualize(atom_group):
    mda_view = nv.show_mdanalysis(atom_group)
    return mda_view.display()

visualize(u.atoms)

NGLWidget(max_frame=9)

## Interactive visualization

To kick off the interactive visualization workflow, we need to find something we are interested in visualizing! Let's see which atoms EA, FEC, and PF6<sub>6</sub><sup>-</sup> tend to coordinate with. We will start with EA.

First lets find out which EA solvation shell composition is most common.

In [78]:
# return the percentage of each shell
solution.speciation.speciation_percent

,EA,FEC,PF6,count
0,3,1,0,0.254098
1,2,3,0,0.168852
2,2,2,0,0.137705
3,1,3,0,0.091803
4,4,0,0,0.057377
5,1,4,0,0.055738
6,3,2,0,0.052459
7,1,2,1,0.031148
8,1,3,1,0.024590
9,0,4,0,0.022951


Looks like 3 EA and 1 FEC is the most common shell. Let's find one to visualize!

In [80]:
# find all shells with 5 BN and nothing else
solution.speciation.find_shells({'EA': 3, 'FEC': 1, 'PF6': 0})

res_name             EA  FEC  PF6
frame solvated_atom              
0     3               3    1    0
      10              3    1    0
      16              3    1    0
      20              3    1    0
      21              3    1    0
...                  ..  ...  ...
9     40              3    1    0
      42              3    1    0
      43              3    1    0
      48              3    1    0
      57              3    1    0

[155 rows x 3 columns]

I like low numbers, so let's choose the third solvent in the zeroth frame. We'll save the AtomGroup as `shell` and just to be safe we'll print the shell DataFrame to make sure we have the right composition

In [81]:
# save the AtomGroup
shell = solution.solvation_shell(solute_index=3, frame=0)

# return the DataFrame
solution.solvation_shell(3, 0, as_df=True)

,dist,res_name,res_ix
atom_ix,,,
2315,1.943975,EA,165
1223,1.997783,EA,87
3141,2.054177,EA,224
3320,2.265687,FEC,238


Now we are all ready to take a look! 

In [82]:
visualize(shell)

NGLWidget(max_frame=9)

Easy! Now we can save the visualization or explore further.

There appears to be a space to one side of our Li ion, lets see what's there by using another selection function.

In [87]:
u.trajectory[0]
shell_5 = solution.closest_n_mol(solute_index=3, n_mol=5)

In [88]:
visualize(shell_5)

NGLWidget(max_frame=9)

Looks like an uncoordinate EA molecule is filling that space. 

We'll stop there, but hopefully this provides a taste of what interactive visualization can look like!